# Adjacency Matrix for Order Preservation

This notebook demonstrates how Adjacency Matrices (AM) preserve token order and enable reconstruction during query processing.

**Key Architecture**:
- **Ingestion**: Single HLLSet combines all n-tokens (1-tokens + 2-tokens + 3-tokens)
- **AM Matrix**: Splits this HLLSet by (reg, zeros) identifiers for disambiguation
- **~100K × 100K matrix**: Much smaller than millions of unique tokens
- **For concise languages** (e.g., Chinese ~80K characters): Can split by actual tokens

**Key Concepts**:
- Build AM during ingestion with transition frequencies
- Use (reg, zeros) identifiers as compact representation
- Traverse AM during queries to reconstruct order
- START/END markers for sequence boundaries

## 1. Understanding the Problem

**HLLSets lose order**:
- Only store distinct tokens
- No sequence information
- Cannot reconstruct "hello world" vs "world hello"

**Solution**: Adjacency Matrix preserves transitions between tokens.

In [1]:
from core.manifold_os import ManifoldOS
from core import HLLSet

# Create ManifoldOS instance
os = ManifoldOS()

# Ingest text - builds AM (single source of truth)
text = "the quick brown fox jumps over the lazy dog"
result = os.ingest(text)

# Access the adjacency matrix from the IngestDriver
driver = os.get_driver("ingest_default")
am = driver.adjacency_matrix

print(f"Text ingested: '{text}'")
print(f"\n📊 Adjacency Matrix (Source of Truth):")
print(f"  Non-zero cells (transitions): {am.get_nonzero_count()}")
print(f"  Matrix size: {am.get_size()[0]} rows × {am.get_size()[1]} cols")
print(f"  Row HLLSets: {len(am.row_hllsets)}")
print(f"  Column HLLSets: {len(am.col_hllsets)}")

print(f"\n🔍 HLLSet derived from AM (before commit):")
print(f"  Complete HLLSet extracted: {result.complete_hllset is not None}")
if result.complete_hllset:
    print(f"  Cardinality: {result.complete_hllset.cardinality():.2f}")
    print(f"  ✓ Derived from union of AM's row HLLSets")
    print(f"  ✓ Extracted BEFORE AM committed to shared resource")

print(f"\n💡 Architecture:")
print(f"  • AM = Single source of truth (order + sets)")
print(f"  • HLLSet = Derived from AM during ingestion")
print(f"  • After commit: AM merges into shared (can't derive individual HLLSets)")


✓ Extension registered: storage v1.4.4
✓ Extension registered: storage v1.4.4
  ✓ LUT committed: n=1, hash=dbe1e60d59b56929..., id=8
  ✓ LUT committed: n=2, hash=fdb8b90c523da265..., id=8
  ✓ LUT committed: n=3, hash=e4545c77cb6272c7..., id=7
Text ingested: 'the quick brown fox jumps over the lazy dog'

📊 Adjacency Matrix (Source of Truth):
  Non-zero cells (transitions): 10
  Matrix size: 9 rows × 9 cols
  Row HLLSets: 9
  Column HLLSets: 9

🔍 HLLSet derived from AM (before commit):
  Complete HLLSet extracted: True
  Cardinality: 12.00
  ✓ Derived from union of AM's row HLLSets
  ✓ Extracted BEFORE AM committed to shared resource

💡 Architecture:
  • AM = Single source of truth (order + sets)
  • HLLSet = Derived from AM during ingestion
  • After commit: AM merges into shared (can't derive individual HLLSets)


## 2. Sliding Window Processing

During ingestion, tokens are processed with a sliding window to build transitions.

In [2]:
from core.manifold_os import ManifoldOS

# Create OS
os = ManifoldOS()

# Ingest a sentence - AM is built automatically
text = "the quick brown fox jumps over the lazy dog"
result = os.ingest(text)

print("Original tokens:", result.original_tokens)
print(f"\nHLLSets created: {len(result.hllsets)}")
print("\nSliding window creates transitions:")
print("  'the' → 'quick'")
print("  'quick' → 'brown'")
print("  'brown' → 'fox'")
print("  ...")

✓ Extension registered: storage v1.4.4
✓ Extension registered: storage v1.4.4
  ✓ LUT committed: n=1, hash=dbe1e60d59b56929..., id=8
  ✓ LUT committed: n=2, hash=fdb8b90c523da265..., id=8
  ✓ LUT committed: n=3, hash=e4545c77cb6272c7..., id=7
Original tokens: ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']

HLLSets created: 3

Sliding window creates transitions:
  'the' → 'quick'
  'quick' → 'brown'
  'brown' → 'fox'
  ...


## 3. (reg, zeros) Identifiers

Instead of storing full token strings, AM uses compact identifiers derived from hash values.

In [3]:
from core import HLLSet
from core.constants import SHARED_SEED

# Now we use HLLSet's efficient batch method - no double calculation!
tokens = ['the', 'quick', 'brown', 'fox']

# Get (reg, zeros) identifiers efficiently
identifiers = HLLSet.compute_reg_zeros_batch(tokens, seed=SHARED_SEED)

print("Token identifiers (reg, zeros):\n")
for token, (reg, zeros) in zip(tokens, identifiers):
    print(f"  '{token}' → (reg={reg}, zeros={zeros})")

print("\n✓ Single hash calculation in C backend")
print("✓ Used by both HLLSet and Adjacency Matrix")
print("✓ No duplicate work!")

Token identifiers (reg, zeros):

  'the' → (reg=960, zeros=1)
  'quick' → (reg=892, zeros=2)
  'brown' → (reg=573, zeros=0)
  'fox' → (reg=678, zeros=3)

✓ Single hash calculation in C backend
✓ Used by both HLLSet and Adjacency Matrix
✓ No duplicate work!


## 4. START and END Markers

Special markers define sequence boundaries.

In [4]:
# Access the adjacency matrix to see START/END tokens
driver = os.get_driver("ingest_default")
am = driver.adjacency_matrix

print("Special token identifiers:")
print(f"  START → {am.START_ID}")
print(f"  END → {am.END_ID}")

# Show actual transitions with START/END
print("\nActual transitions in AM (showing START/END):")
for (from_id, to_id), cell in list(am.cells.items())[:6]:
    from_label = "START" if from_id == am.START_ID else str(from_id)
    to_label = "END" if to_id == am.END_ID else str(to_id)
    print(f"  {from_label} → {to_label} (freq={cell.frequency})")

print("\n✓ START marks sequence beginning")
print("✓ END marks sequence termination")
print("✓ Enables order reconstruction during queries")

Special token identifiers:
  START → (-1, 0)
  END → (-2, 0)

Actual transitions in AM (showing START/END):
  START → (960, 1) (freq=1)
  (960, 1) → (892, 2) (freq=1)
  (892, 2) → (573, 0) (freq=1)
  (573, 0) → (678, 3) (freq=1)
  (678, 3) → (956, 6) (freq=1)
  (956, 6) → (648, 3) (freq=1)

✓ START marks sequence beginning
✓ END marks sequence termination
✓ Enables order reconstruction during queries


## 5. Transition Frequencies

AM stores how often token A transitions to token B.

In [5]:
# Simulate building transitions
from collections import defaultdict

# Sample text with repetitions
text = "the cat sat on the mat and the cat sat"
tokens = text.split()

# Count transitions
transitions = defaultdict(int)
for i in range(len(tokens) - 1):
    pair = (tokens[i], tokens[i+1])
    transitions[pair] += 1

print("Transition frequencies:\n")
for (from_token, to_token), count in sorted(transitions.items(), key=lambda x: -x[1]):
    print(f"  '{from_token}' → '{to_token}': {count}")

Transition frequencies:

  'the' → 'cat': 2
  'cat' → 'sat': 2
  'sat' → 'on': 1
  'on' → 'the': 1
  'the' → 'mat': 1
  'mat' → 'and': 1
  'and' → 'the': 1


## 6. Query Phase: Order Reconstruction

During queries, traverse the AM to reconstruct original order.

In [6]:
def reconstruct_order(hllset, threshold=0.9):
    """
    Reconstruct order from AM (conceptual demonstration).
    
    Algorithm:
    1. Start at START marker
    2. Follow highest frequency transitions
    3. Stop when reaching threshold * cardinality tokens
    4. Or when reaching END marker
    
    Note: This is a simplified conceptual demo.
    Real implementation would query the actual AM transitions.
    """
    cardinality = hllset.cardinality()
    target_tokens = int(cardinality * threshold)
    
    print(f"HLLSet cardinality: {cardinality:.0f}")
    print(f"Target: ~{target_tokens} tokens ({threshold*100}% of cardinality)")
    print("\nConceptual traversal: START", end="")
    
    # Simplified traversal visualization
    reconstructed = []
    current = "START"
    
    # In real implementation, would look up transitions from AM
    # For demo, just show the concept
    sample_path = ["the", "quick", "brown", "fox", "jumps"]
    for token in sample_path[:min(len(sample_path), target_tokens)]:
        print(f" → {token}", end="")
        reconstructed.append(token)
    
    print(" → END")
    return reconstructed

# Demo with a sample HLLSet
demo_hll = HLLSet.from_batch(['the', 'quick', 'brown', 'fox', 'jumps'])
reconstruct_order(demo_hll, threshold=0.9)

HLLSet cardinality: 7
Target: ~6 tokens (90.0% of cardinality)

Conceptual traversal: START → the → quick → brown → fox → jumps → END


['the', 'quick', 'brown', 'fox', 'jumps']

## 7. Two-Phase Architecture

Understanding how AM works across ingestion and query phases.

In [7]:
print("="*70)
print("PHASE 1: INGESTION (Building AM)")
print("="*70)
print("""
1. Tokenize input text
2. Generate n-tokens via sliding window (1-tokens, 2-tokens, 3-tokens)
3. Create SINGLE HLLSet combining all n-tokens
4. For AM matrix:
   - Split HLLSet by (reg, zeros) identifiers
   - Or by actual tokens for concise languages (Chinese ~80K)
5. For each adjacent pair:
   - Get (reg, zeros) identifiers
   - Increment AM[from_id][to_id]
6. Add START → first_token
7. Add last_token → END
8. HLLSet = perfect fingerprint (full set operations support)
""")

print("="*70)
print("PHASE 2: QUERY (Order Reconstruction)")
print("="*70)
print("""
1. User provides prompt → create HLLSet
2. Retrieve relevant HLLSets from storage
3. Get cardinality estimate (note: tripled due to all n-tokens)
4. Traverse AM:
   - Start at START marker
   - Follow highest frequency transitions
   - Collect identifiers
   - Stop when threshold reached (e.g., 0.9 × cardinality)
   - Or when END marker reached
5. Map identifiers back to tokens via LUT
6. Return ordered sequence
""")

PHASE 1: INGESTION (Building AM)

1. Tokenize input text
2. Generate n-tokens via sliding window (1-tokens, 2-tokens, 3-tokens)
3. Create SINGLE HLLSet combining all n-tokens
4. For AM matrix:
   - Split HLLSet by (reg, zeros) identifiers
   - Or by actual tokens for concise languages (Chinese ~80K)
5. For each adjacent pair:
   - Get (reg, zeros) identifiers
   - Increment AM[from_id][to_id]
6. Add START → first_token
7. Add last_token → END
8. HLLSet = perfect fingerprint (full set operations support)

PHASE 2: QUERY (Order Reconstruction)

1. User provides prompt → create HLLSet
2. Retrieve relevant HLLSets from storage
3. Get cardinality estimate (note: tripled due to all n-tokens)
4. Traverse AM:
   - Start at START marker
   - Follow highest frequency transitions
   - Collect identifiers
   - Stop when threshold reached (e.g., 0.9 × cardinality)
   - Or when END marker reached
5. Map identifiers back to tokens via LUT
6. Return ordered sequence



## 8. Compact Matrix Size

**AM splits single HLLSet by identifiers for compact representation**:

- **Option 1**: (reg, zeros) identifiers → ~100K × 100K matrix
- **Option 2**: Actual tokens for concise languages (e.g., Chinese ~80K characters)

Both approaches much smaller than millions of unique tokens in full vocabulary.

In [8]:
# Calculate matrix dimensions for (reg, zeros) approach
p_bits = 10
max_zeros = 32

# Number of unique identifiers
num_regs = 2 ** p_bits  # 1024 registers
num_identifiers = num_regs * (max_zeros + 1)  # 1024 × 33 = 33,792

print("Option 1: (reg, zeros) identifiers")
print(f"  P_BITS: {p_bits}")
print(f"  Registers: {num_regs:,}")
print(f"  Max zeros: {max_zeros}")
print(f"  Total identifiers: {num_identifiers:,}")
print(f"  Matrix size: {num_identifiers:,} × {num_identifiers:,}")

print("\nOption 2: Actual tokens (concise languages)")
print(f"  Chinese: ~80,000 characters")
print(f"  Matrix size: 80,000 × 80,000")

print(f"\nVs. millions of unique tokens in full vocabulary")
print(f"✓ Single HLLSet combines all n-tokens (perfect fingerprint)")
print(f"✓ AM splits it by identifiers for compact storage")

# Memory calculation (sparse matrix)
entries_per_mb = 1024 * 1024 / 8  # 8 bytes per entry
print(f"\nWith sparse storage, only non-zero transitions stored")

Option 1: (reg, zeros) identifiers
  P_BITS: 10
  Registers: 1,024
  Max zeros: 32
  Total identifiers: 33,792
  Matrix size: 33,792 × 33,792

Option 2: Actual tokens (concise languages)
  Chinese: ~80,000 characters
  Matrix size: 80,000 × 80,000

Vs. millions of unique tokens in full vocabulary
✓ Single HLLSet combines all n-tokens (perfect fingerprint)
✓ AM splits it by identifiers for compact storage

With sparse storage, only non-zero transitions stored


## 9. AM as Shared Resource - Timing is Critical

**Key Architectural Insight**: AM is a shared resource that gets committed/merged after ingestion.

In [9]:
from core.manifold_os import TokenizationConfig, IngestDriver

print("TIMING DEMONSTRATION:")
print("=" * 70)

# Test 1: WITH HLLSet extraction (default)
print("\n1️⃣  During Ingestion (extract_hllset_from_am=True):")
print("   ┌─ Build AM from tokens")
print("   ├─ Extract HLLSet from AM (BEFORE commit)")
print("   └─ Commit AM → merges into shared resource")

os1 = ManifoldOS()
result1 = os1.ingest("hello world")
print(f"\n   Result: complete_hllset = {result1.complete_hllset is not None}")
print(f"   ✓ HLLSet extracted in time!")

# Test 2: WITHOUT HLLSet extraction
print("\n2️⃣  Skip Extraction (extract_hllset_from_am=False):")
print("   ┌─ Build AM from tokens")
print("   ├─ SKIP HLLSet extraction")
print("   └─ Commit AM → merges into shared resource")

config = TokenizationConfig(extract_hllset_from_am=False)
os2 = ManifoldOS()
driver = IngestDriver("no_extract", config=config)
os2.register_driver(driver)
driver.wake()

result2 = os2.ingest("hello world", driver_id="no_extract")
print(f"\n   Result: complete_hllset = {result2.complete_hllset is not None}")
print(f"   ⚠️  No HLLSet (extraction skipped)")

print("\n" + "=" * 70)
print("💡 Key Lesson: Extract HLLSet DURING ingestion, BEFORE commit!")
print("   After commit, AM is shared - can't derive individual HLLSets")


TIMING DEMONSTRATION:

1️⃣  During Ingestion (extract_hllset_from_am=True):
   ┌─ Build AM from tokens
   ├─ Extract HLLSet from AM (BEFORE commit)
   └─ Commit AM → merges into shared resource
✓ Extension registered: storage v1.4.4
✓ Extension registered: storage v1.4.4
  ✓ LUT committed: n=1, hash=c2b76b7f6c7389f3..., id=2
  ✓ LUT committed: n=2, hash=1915bbc8b2e9217f..., id=1
  ✓ LUT committed: n=3, hash=1ceaf73df40e531d..., id=0

   Result: complete_hllset = True
   ✓ HLLSet extracted in time!

2️⃣  Skip Extraction (extract_hllset_from_am=False):
   ┌─ Build AM from tokens
   ├─ SKIP HLLSet extraction
   └─ Commit AM → merges into shared resource
✓ Extension registered: storage v1.4.4
✓ Extension registered: storage v1.4.4
  ✓ LUT committed: n=1, hash=c2b76b7f6c7389f3..., id=2
  ✓ LUT committed: n=2, hash=1915bbc8b2e9217f..., id=1
  ✓ LUT committed: n=3, hash=1ceaf73df40e531d..., id=0

   Result: complete_hllset = False
   ⚠️  No HLLSet (extraction skipped)

💡 Key Lesson: Extract H

## Summary

**Adjacency Matrix Benefits**:

1. **Order Preservation**: Captures token transitions
2. **Compact Size**: ~100K × 100K vs millions of tokens
3. **Query Support**: Traverse to reconstruct order
4. **Frequency Tracking**: Higher frequencies = more common transitions
5. **START/END**: Clear sequence boundaries

**Workflow**:
- **Ingestion**: Build AM with transitions
- **Query**: Traverse AM to reconstruct order
- **LUT**: Map identifiers back to original tokens

---

## HLLSet Paradigm Shift: Freedom of Representation

**Traditional Data Structures** operate with a "copy and paste" mindset:
- Data must be stored in specific formats
- Transformations create separate copies
- Identity means exact structural match

**HLLSet Algebra** offers transformational freedom:
- **Same dataset** = any equal transformation
- Single HLLSet combining all n-tokens ≡ Separate HLLSets per group
- (reg, zeros) identifiers ≡ actual tokens (for concise languages)
- **Freedom**: Choose the representation that best serves your use case

**Examples of Equivalent Representations**:
```python
# These are all equivalent "views" of the same data:
hll_combined = HLLSet.from_batch(tokens_1 + tokens_2 + tokens_3)
hll_separate = hll_1.union(hll_2).union(hll_3)
# Both have identical cardinality and support same operations

# AM can split by:
am_identifiers = build_am(hll, key=lambda t: (reg(t), zeros(t)))  # Compact
am_tokens = build_am(hll, key=lambda t: t)                        # Explicit
# Choice depends on matrix size vs token vocabulary
```

**Learning HLLSet Algebra**:
- Operations: union, intersection, difference, similarity
- Transformations preserve cardinality relationships
- No forced "canonical" representation
- Algebra guides you to optimal representations

This flexibility is **powerful**: choose representations that optimize for your specific needs (memory, speed, interpretability) while maintaining mathematical equivalence.

**Next**: See [04_kernel_entanglement.ipynb](04_kernel_entanglement.ipynb) for kernel operations.